In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [3]:
src_path = '../data/AG/100nt/'
file_types = ('.txt')
 
file_paths = []  
for root, dirs, files in os.walk(src_path):
    file_paths.extend([os.path.join(root, f) for f in files if f.endswith(file_types)])
    
print 'number of files is', len(file_paths)

number of files is 3


In [4]:
file_paths

['../data/AG/100nt/ag-cleaned_L2.txt',
 '../data/AG/100nt/ag-cleaned_L3.txt',
 '../data/AG/100nt/ag-cleaned_L6.txt']

In [5]:
targets = ["BMI_CORRECTED", "AGE_CORRECTED"]

biomarker = targets[0]

In [ ]:
RF_mean_scores = []
RF_std_scores = []
KNN_mean_scores = []
KNN_std_scores = []

for file_path in file_paths:
    df = pd.read_table(file_Path)
    df_rel = df[df["HMP_SITE"] == "FECAL"]
    df_rel[[biomarker]] = df_rel[[biomarker]].apply(lambda x: pd.to_numeric(x, errors = "coerce"))
    df_clean = df_rel.dropna()

    y = np.array(df_clean[biomarker])
    X = np.array(df_clean.iloc[:,467:])
    print "y shape", y.shape
    print "X shape", X.shape
    
    
    X_train = X[:int(.8*X.shape[0]),:] # use FIRST 80% of data as training set to test on 'future'
    y_train = y[:int(.8*X.shape[0])]
    X_test = X[int(.8*X.shape[0])+1:,:]
    y_test = y[int(.8*X.shape[0])+1:]


    RF = RandomForestRegressor(n_estimators=1000)
    scores = -1*cross_validation.cross_val_score(RF, X, y, cv=10, scoring = 'mean_absolute_error')
    RF_mean_scores.append(np.mean(scores))
    RF_std_scores.append(np.std(scores))
    
    #KNN =  KNeighborsRegressor(n_neighbors=5)
    KNN = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=dict(n_neighbors=range(1,11)), n_jobs = -1, scoring = 'mean_absolute_error')
    scores = -1*cross_validation.cross_val_score(KNN, X, y, cv=10, scoring = 'mean_absolute_error')
    KNN_mean_scores.append(np.mean(scores))
    KNN_std_scores.append(np.std(scores))    
    
print "average y is:", np.mean(y)
print "std y is:", np.std(y)

In [ ]:
width = 12
height = 6
fig = plt.figure(figsize=(width, height))
ax = fig.add_subplot(111)
ax.set_title(biomarker)
ax.set_xlabel('Taxonomic classification level')
ax.set_ylabel('Mean absolute error (lower is better)')

plt.errorbar(range(5), RF_mean_scores, yerr=np.array(RF_std_scores)/5., marker='s', c = "b", label = "RF", fmt='o')
plt.errorbar(range(5), KNN_mean_scores, yerr=np.array(KNN_std_scores)/5., marker='o', c = "r", label = "KNN", fmt='o')
plt.legend(loc='upper left')
plt.xticks(range(5), ['L2: Phylum', 'L3: Class', 'L6: Genus'])